# Otimização: Método de Newton e Quasi-Newton

Minimizar funções do tipo $f\left(x\right)=exp({\sum} _{i=1}^{n} x_{i}) +\sum _{i=1}^{n} a_{i} x_{i}^{2} $. 

Para isso é utilizado o [método de Newton](https://en.wikipedia.org/wiki/Newton%27s_method), e também o método de [Barzilai-Browein](http://www.math.ucla.edu/~wotaoyin/math273a/slides/Lec4a_Baizilai_Borwein_method_273a_2015_f.pdf), que é um método [Quasi-Newton](https://en.wikipedia.org/wiki/Quasi-Newton_method).

### Dependências

In [30]:
import numpy as np
import scipy as sp
from scipy import optimize

### Funções Auxiliares

In [41]:
grad = lambda x: np.exp(np.sum(x))+2*np.multiply(A,x)
apply_func = lambda x: np.exp(np.sum(x))+np.dot(A,np.multiply(x,x))
hess = lambda x: 2*np.exp(np.sum(x))+np.diag(A)
d_func = lambda h,g: -np.linalg.inv(h).dot(g)
close_to_zero = lambda d: np.dot(d,d) <= 1e-8

### Funções Principais

**Método de Newton:**

In [67]:
def newton(A, x, verbose = False):
    
    d = d_func(hess(x),grad(x))
    k = 1
    

    while(close_to_zero(d) == False):
        alpha = sp.optimize.line_search(apply_func,grad,x,d)[0]
        x = x + alpha*d
        d = d_func(hess(x),grad(x))
        
        if(verbose == True):
            print('x = ', x, ' ( It ', k, ')')
            
        k += 1
                
    return x, d

Testando a função:

In [68]:
A = np.array([42,10,11])
x, d = newton(A, x = np.array([0,0,0]), verbose=False)
print('---------------------------')
print('x final = ', x)
print('d final = ', d)

---------------------------
x final =  [-0.01080852 -0.04539577 -0.04126888]
d final =  [1.35149384e-05 5.67627413e-05 5.16024921e-05]


**Barzilai-Browein:**

In [69]:
def barzilai_browein(A, x, verbose = False):
    
    g = grad(x)
    d = -g
    alpha = sp.optimize.line_search(apply_func,grad,x,d)[0]
    k = 1
    
    while(close_to_zero(d) == False):
        next_x = x + alpha*d
        next_g = grad(next_x)
        s, y = (next_x - x),(next_g - g)
        alpha = s.dot(y) / y.dot(y)
        g = next_g
        x = next_x
        d = -g
        
        if(verbose == True):
            print('x = ', x, ' ( It ', k, ')')
            
        k += 1
                   
    return x, d

Testando a função:

In [70]:
A = np.array([42,10,11])
x, d = barzilai_browein(A, x = np.array([0,0,0]), verbose=False)
print('---------------------------')
print('x final = ', x)
print('d final = ', d)

---------------------------
x final =  [-0.01079996 -0.04535986 -0.04123628]
d final =  [-5.99220686e-08  5.89273975e-07  1.66642889e-06]


## Experimentos

Primeira com valores relativamente próximos:

- Newton com valores próximos

In [71]:
A = np.array([5,2,4])
x, d = newton(A, x = np.array([0,0,0]), verbose=True)
print('---------------------------')
print('x final = ', x)
print('d final = ', d)

x =  [-0.06896552 -0.17241379 -0.0862069 ]  ( It  1 )
x =  [-0.07158288 -0.17895719 -0.0894786 ]  ( It  2 )
x =  [-0.07123669 -0.17809173 -0.08904586]  ( It  3 )
x =  [-0.07128438 -0.17821096 -0.08910548]  ( It  4 )
---------------------------
x final =  [-0.07128438 -0.17821096 -0.08910548]
d final =  [6.53626879e-06 1.63406720e-05 8.17033599e-06]


- Barzilai Browein com valores próximos

In [72]:
A = np.array([5,2,4])
x, d = barzilai_browein(A, x = np.array([0,0,0]), verbose=True)
print('---------------------------')
print('x final = ', x)
print('d final = ', d)

x =  [-0.1149444 -0.1149444 -0.1149444]  ( It  1 )
x =  [-0.07293882 -0.13861435 -0.09483066]  ( It  2 )
x =  [-0.07361397 -0.15687551 -0.09256456]  ( It  3 )
x =  [-0.07109103 -0.17680955 -0.08913687]  ( It  4 )
x =  [-0.07174379 -0.17826888 -0.08930977]  ( It  5 )
x =  [-0.07108296 -0.17816402 -0.08902615]  ( It  6 )
x =  [-0.07127926 -0.17819517 -0.0890977 ]  ( It  7 )
---------------------------
x final =  [-0.07127926 -0.17819517 -0.0890977 ]
d final =  [ 5.27024828e-06 -6.69336224e-06 -5.72899100e-06]


E agora com valores mais discrepantes:
    
- Newton com valores discrepantes

In [73]:
A = np.array([10000,10,1])
x, d = newton(A, x = np.array([0,0,0]), verbose=True)
print('---------------------------')
print('x final = ', x)
print('d final = ', d)

x =  [-3.1248047e-05 -3.1248047e-02 -3.1248047e-01]  ( It  1 )
x =  [-3.45344944e-05 -3.45344944e-02 -3.45344944e-01]  ( It  2 )
x =  [-3.42642233e-05 -3.42642233e-02 -3.42642233e-01]  ( It  3 )
x =  [-3.4291015e-05 -3.4291015e-02 -3.4291015e-01]  ( It  4 )
---------------------------
x final =  [-3.4291015e-05 -3.4291015e-02 -3.4291015e-01]
d final =  [2.61924322e-09 2.61924322e-06 2.61924322e-05]


- Barzilai-browein com valores discrepantes

In [74]:
A = np.array([10000,10,1])
x, d = barzilai_browein(A, x = np.array([0,0,0]), verbose=True)
print('---------------------------')
print('x final = ', x)
print('d final = ', d)

x =  [-0.00014977 -0.00014977 -0.00014977]  ( It  1 )
x =  [-4.98529832e-05 -1.99657892e-04 -1.99792851e-04]  ( It  2 )
x =  [-4.99775740e-05 -2.49449420e-04 -2.49764107e-04]  ( It  3 )
x =  [-4.91581929e-05 -8.35569461e-03 -8.39260463e-03]  ( It  4 )
x =  [-5.84764598e-05 -5.08524061e-02 -5.87161332e-02]  ( It  5 )
x =  [ 0.01357215 -0.04482496 -0.09754636]  ( It  6 )
x =  [-6.05639055e-05 -4.48240917e-02 -9.75806025e-02]  ( It  7 )
x =  [-4.33616352e-05 -4.48226286e-02 -9.76142038e-02]  ( It  8 )
x =  [-4.33602358e-05 -4.48211654e-02 -9.76478193e-02]  ( It  9 )
x =  [-3.35069996e-05 -3.44802325e-02 -3.35210880e-01]  ( It  10 )
x =  [-0.00760491 -0.03496105 -0.34267964]  ( It  11 )
x =  [-3.32208706e-05 -3.49601061e-02 -3.42679383e-01]  ( It  12 )
x =  [-3.42727044e-05 -3.49594187e-02 -3.42679388e-01]  ( It  13 )
x =  [-3.42727277e-05 -3.49587317e-02 -3.42679393e-01]  ( It  14 )
x =  [-3.42952999e-05 -3.42961661e-02 -3.42684040e-01]  ( It  15 )
x =  [-3.43015551e-05 -3.42953357e-02 -3

## Conclusão

Através dos experimentos conduzidos, podemos observar que há uma considerável diferença entre as iterações necessárias para os métodos (de Newton e Barzilai-browein) convergirem. Em particular, observa-se que o método de newton necessita de menos iterações para convergir.

Todavia, é importante notar que a computação requerida para o método de newton é superior à computação requerida por Barzilai-browein e, desta forma, para problemas de maior escala pode ser preferível a aplicação do Barzilai-browein em relação ao método de Newton.

O Newton propociona convergência quadrática, o que é melhor que a taxa de convergência dos métodos de gradiente (que possuem convergência linear).